# Семинар по теме «Шифрование и стеганография»

<!-- Закрепим знания, полученные при изучении тем «Шифрование» и «Стеганография». -->

## Оценивание работы на семинаре

**Система оценивания —** бинарная:

  - если все задачи решены корректно, без ошибок и полностью соответствуют стандартам кода на курсе, то задание выполнено и оценка — **10 баллов**;
  - если решения содержат ошибки или не соответствуют требованиям, то задание не выполнено и оценка — **0 баллов**.  


**Проверка задания**

- Перед тем, как сдать задание, убедись, что твой код работает без ошибок и соответствует стандартам.
- Загрузи задание на LMS. Ассистент проверит, соответствуют ли твои решения требованиям и целям домашнего задания, и выставит оценку.


**Доработка**

- Если твое задание получило 0 баллов, его вернут на доработку через LMS с комментариями о том, что нужно исправить.

При работе со стеганографией важно уметь оценивать, как скрытие данных влияет на качество изображения. Это можно делать визуально, а можно — с помощью метрик, например среднеквадратической ошибки (**MSE**, англ. mean squared error) или пикового отношения сигнал/шум (**PSNR**, англ. peak signal-to-noise ratio).


### Среднеквадратическая ошибка (MSE)

Среднеквадратичная ошибка измеряет среднее значение квадратов разности между соответствующими пикселями двух изображений. Она показывает, насколько сильно изменились пиксели после кодирования данных. Чем меньше значение MSE, тем меньше изменение и тем ближе закодированное изображение к оригиналу. MSE вычисляется по формуле:
$$
\text{MSE} = \frac{1}{MN} \sum_{i=1}^{M} \sum_{j=1}^{N} \left[ I(i,j) - K(i,j) \right]^2,
$$

где:
- $M$ и $N$ — размеры изображения (высота и ширина в пикселях);
- $I(i,j)$ — значение пикселя в позиции $(i, j)$ в оригинальном изображении;
- $K(i,j)$ — значение пикселя в позиции $(i, j)$ в закодированном изображении.

**Как посчитать:**

1. Преобразуй оба изображения в массивы пикселей.
2. Найди разницу между соответствующими пикселями в двух изображениях.
3. Возведи разницу в квадрат и сложи все такие значения.
4. Раздели полученную сумму на общее количество пикселей.

Пример кода для вычисления MSE:



```python
from PIL import Image

def calculate_mse(image1, image2):
    # Открываем изображения
    img1 = Image.open(image1)
    img2 = Image.open(image2)

    # Проверяем, что размеры изображений совпадают
    if img1.size != img2.size:
        raise ValueError('Изображения должны быть одинакового размера')

    # Получаем размеры изображений
    width, height = img1.size

    # Инициализируем переменную для суммы квадратов разностей
    total_error = 0

    # Проходим по каждому пикселю
    for x in range(width):
        for y in range(height):
            # Получаем значение пикселя для каждого изображения (в формате RGB)
            pixel1 = img1.getpixel((x, y))
            pixel2 = img2.getpixel((x, y))

            # Считаем разницу по каждому цветовому каналу (R, G, B)
            for i in range(3):  # R, G, B
                diff = pixel1[i] - pixel2[i]
                total_error += diff ** 2

    # Считаем общее количество пикселей
    num_pixels = width * height * 3  # умножаем на 3, так как у нас три канала (R, G, B)

    # Возвращаем среднеквадратичную ошибку
    mse = total_error / num_pixels
    return mse

# Пример использования:
mse_value = calculate_mse('original_image.png', 'encoded_image.png')
print(f'MSE: {mse_value}')
```




### Пиковое отношение сигнал/шум (PSNR)

Эта метрика оценивает качество изображения, выраженное в децибелах (дБ). PSNR зависит от значения MSE и максимального возможного значения пикселя (обычно 255 для изображений с 8-битной глубиной цвета). Чем выше значение PSNR, тем лучше качество изображения. Вычисляется по формуле:

$$
\text{PSNR} = 20 \cdot \log_{10} \left(\frac{MAX_I}{\sqrt{\text{MSE}}}\right)\!,
$$

где:
- $MAX_I$ — максимальное значение пикселя (например, 255 для 8-битных изображений).
- $\text{MSE}$ — среднеквадратичная ошибка между оригинальным и закодированным изображением.


> В PSNR используется логарифм с основанием 10, потому что это стандарт при работе с децибелами — величинами, которые измеряют отношение сигналов.
Стандартная формуля для децибелов выглядит так:
$$
10 \cdot \log_{10}\frac{P_2}{P_1}.
$$
Множитель 20 появляется, потому что мы измеряем не мощность $P$ (которая пропорциональна квадрату разности), а амплитуду $A$ (пропорциональную разности значений):
$$
10 \cdot \log_{10}\frac{P_\text{сигнал}}{P_\text{шум}}= 10 \cdot \log_{10}\frac{A^2_\text{сигнал}}{A^2_\text{шум}}=20 \cdot \log_{10}\frac{A_\text{сигнал}}{A_\text{шум}}.
$$
<!-- В таких случаях коэффициент перед логарифмом удваивается, и вместо 10 мы используем 20. -->


**Как посчитать:**

1. Вычисли MSE между двумя изображениями.
2. Найди максимальное значение пикселя (например, 255).
3. Подставь значения в формулу и вычисли PSNR.

Пример кода для вычисления PSNR:


```python
import math

def calculate_psnr(mse, max_pixel=255):
    if mse == 0:
        return float('inf')  # идеальное совпадение изображений
    psnr = 20 * math.log10(max_pixel / math.sqrt(mse))
    return psnr

# Используем ранее вычисленное значение MSE
psnr_value = calculate_psnr(mse_value)
print(f'PSNR: {psnr_value} dB')
```

---

**Значения показателей MSE**

|        |        |
|--------|--------|
|**MSE**|**Значение**|
| ≈ 0 | Закодированное изображение практически идентично оригиналу  |
| < 100 |Отличия незаметны  |
|от 100 до 1000 | Отличия становятся более заметными, особенно при более высоких значениях MSE  |
|> 1000 | Отличия заметны невооружённым глазом  |

<!-- - **MSE ≈ 0:**  
  Закодированное изображение практически идентично оригиналу.  

- **MSE < 100:**  
  Есть незначительные отличия, которые, скорее всего, незаметны человеческому глазу.  

- **MSE от 100 до 1000:**  
  Отличия становятся более заметными, особенно при более высоких значениях MSE.  

- **MSE > 1000:**  
  Отличия заметны невооруженным глазом.   -->

**Значения показателей PSNR**

|        |        |
|--------|--------|
|**MSE**|**Значение**|
| > 40 дБ | Закодированное изображение практически идентично оригиналу  |
| от 30 до 40 дБ |Хорошее качество изображения, отличия минимальны |
|от 20 до 30 дБ | Приемлемое качество изображения, но можно заметить отличия, особенно на однородных участках  |
| < 20 дБ | Изображение значительно отличается от оригинала  |

<!-- - **PSNR > 40 дБ:**  
  Изображение практически не отличается от оригинала, изменения минимальны и не заметны для человеческого глаза.  

- **PSNR от 30 до 40 дБ:**  
  Хорошее качество изображения, визуальные изменения минимальны.  

- **PSNR от 20 до 30 дБ:**  
  Качество изображения приемлемое, но изменения могут быть заметны, особенно на однородных участках.  

- **PSNR < 20 дБ:**  
  Изображение значительно отличается от оригинала, изменения явно видны.   -->

---

### **Как использовать метрики для оценки качества кодирования**

После того как посчитаешь MSE и PSNR, сравни их со значениями в таблице, чтобы оценить качество кодирования.

<!-- - Если **MSE** низкое и **PSNR** высокое, значит, кодирование выполнено хорошо, и визуальные изменения минимальны. -->
- Высокая **MSE** и низкое **PSNR**  указывают на необходимость пересмотреть параметры кодирования (например, уменьшить количество бит на пиксель), чтобы улучшить качество изображения.


### Подготовка к выполнению заданий

1. Скачай и распакуй архив `Class04K.zip` и открой проект в IDE.  
2. Создай виртуальное окружение в проекте и активируй его.

# Задача 1

Реализуй стеганографию, при которой используется только один из цветовых каналов (например, красный). Проверь, насколько изменение только одного канала влияет на качество изображения.

<!-- Маркетинговое агентство хочет спрятать текстовое сообщение в изображении, используя только один из цветовых каналов (например, только красный или только синий). Необходимо реализовать такую стеганографию и проверить, насколько изменение только одного канала влияет на изображение. -->

- Создай файл `task_1.py`.
- Запроси у пользователя, какой канал использовать для кодирования данных: красный, зелёный или синий.
- Реализуй функцию `encode_in_channel(image, binary_data, channel)`, которая скрывает данные только в выбранном канале. Она должна принимать следующие аргументы:
  - изображение, открытое библиотекой `Pillow`;
  - бинарные данные для кодирования;
  - название канала для кодирования, полученное от пользователя — строка `'r'`, `'g'` или `'b'`.

- Сравни оригинал и закодированное изображение, чтобы проанализировать, как изменение одного канала влияет на визуальное качество изображения.
- Проанализируй значения метрик MSE и PSNR по одному цветовому каналу.

# Задача 2

Сравни эффективность стеганографии методом LSB для альфа-канала и любого цветного (r, g или b). Для этого используй метрики MSE и PSNR.

- Создай файл `task_2.py`.
- Реализуй функцию `encode_color(image, binary_data)` для стандартного метода LSB, которая принимает аргументы:
  - изображение, открытое библиотекой `Pillow`;
  - бинарные данные для кодирования.

- Для изображения формата PNG с альфа-каналом  реализуй функцию `encode_alpha(image, binary_data)`, которая применяет метод LSB к альфа-каналу.
Она должна принимать следующие аргументы:
  - изображение, открытое библиотекой `Pillow`;
  - бинарные данные для кодирования.

- Сравни визуальное качество изображений после применения каждого метода. Есть ли разница?
- Проанализируй значения метрик MSE и PSNR для обоих случаев.

# Задача 3

>Плотность кодирования — это количество битов данных, которое можно закодировать в каждый пиксель изображения. Чем больше плотность кодирования, тем больше информации можно спрятать в изображении. Однако слишком высокая плотность может привести к заметным искажениям изображения.

Разработчикам нужно определить, сколько информации можно скрыть в изображении незаметно. Твоя задача — автоматически посчитать максимально возможную плотность кодирования, используя метрики MSE и PSNR.

**Задача:**
- Создай файл `task_3.py`.
- Напиши функцию `encode_pixels(image, binary_data, bits)`, которая кодирует данные в изображение с варьируемой плотностью кодирования. Она должна принимать следующие аргументы:
  - изображение, открытое библиотекой `Pillow`;
  - бинарные данные для кодирования;
  - плотность кодирования в битах — целое число от `1` до `8`.


- Напиши функцию `find_max_density(image, binary_data, mse_threshold, psnr_threshold)`, которая перебирает разные значения плотности кодирования и после каждого шага сравнивает изображение с оригиналом по метрикам MSE и PSNR. Она должна принимать следующие аргументы:
  - изображение, открытое библиотекой `Pillow`;
  - бинарные данные для кодирования;
  - порог допустимого значения MSE;
  - порог допустимого значения PSNR.

  Установи пороговые значения MSE и PSNR для остановки процесса кодирования. По умолчанию пороговые значения должны быть установлены в следующие значения:
    - для MSE — 500 единиц,
    - для PSNR — 30 dB.


- Запусти функцию `find_max_density(...)` с изображением и данными, которые нужно закодировать.


- Функция должна выводить одно число — максимальную плотность кодирования (бит на пиксель), при которой изображение остаётся визуально неизменным. Если качество изображения ухудшается (MSE превышает порог или PSNR падает ниже допустимого уровня), процесс проверки останавливается.

Например, если максимальная безопасная плотность кодирования — 5 бит на пиксель, то функция выводит:
  ```text
  5
  ```



# Задача 4

> Битовая плоскость цифрового дискретного сигнала — это набор битов, соответствующих данной позиции бита в каждом из двоичных чисел, представляющих сигнал. Например, 1-я битовая плоскость —
это все первые биты пикселей.

Разработчики шифра экспериментируют с различными битовыми плоскостями, чтобы определить, как скрытие данных в разных битовых плоскостях влияет на изображение и безопасность данных.

- Создай файл `task_4.py`.
- Напиши функцию `encode_in_bit_plane(image, binary_data, bit_plane)`, которая скрывает данные не только в наименее значащем бите (LSB), но и в других битах (например, во втором или третьем). Функция принимает следующие аргументы:
  - изображение, открытое библиотекой `Pillow`;
  - бинарные данные для кодирования;
  - номер бита для кодирования — целое число.

- Закодируй данные в различных битовых плоскостях.


# Задача 5


Разработчикам нужно выделить области на изображении, соответствующие целевому цвету. Напиши программу, которая ищет объекты заданного цвета и обводит их прямоугольной рамкой.

- Создай файл `task_5.py`.
- Напиши функцию `find_colored_objects(image, target_color, tolerance=..., box_color=...)`, которая принимает следующие аргументы:
	- изображение, открытое библиотекой `Pillow`;
	- целевой цвет (в формате RGB-кортежа);
	- допустимое отклонение от целевого цвета со значением по умолчанию — `30`;
	- цвет рамки для подсветки объектов — кортеж из трёх целых чисел со значением красного цвета по умолчанию — `(255, 0, 0)`.
- Функция должна искать на изображении все объекты, цвет которых близок к целевому с учётом допустимого отклонения.
- Объекты должны быть обведены прямоугольными рамками указанного цвета.

